In [2]:
import json
import pathlib
import sys
import csv
import pandas as pd
import copy
import numpy as np
from disagree import metrics

In [2]:
number_of_matches = {
        "bookwolf"     : 0,
        "cliffnotes"   : 0,
        "gradesaver"   : 0,
        "novelguide"   : 0,
        "pinkmonkey"   : 0,
        "shmoop"       : 0,
        "sparknotes"   : 0,
        "thebestnotes" : 0
    }

f = open(pathlib.Path(f"../../alignments/chapter-level-summary-alignments/fixed_chapter_summaries_all_final.jsonl"),
            encoding='utf-8')
for line in f:
    summary = json.loads(line)
    number_of_matches[summary['source']] += 1

for e in number_of_matches:
    print(e, number_of_matches[e])

source = [i for i in number_of_matches if number_of_matches[i] == max(number_of_matches.values())][0]

bookwolf 201
cliffnotes 1805
gradesaver 2092
novelguide 1480
pinkmonkey 1201
shmoop 3678
sparknotes 1845
thebestnotes 319


In [3]:

# df = pd.read_csv("../../evaluation/csv_results/booksum_summaries/line_by_line_section/chapter-comparison-results-all-rouge-1n-lbl.csv")
df = pd.read_parquet("../../evaluation/csv_results/booksum_summaries/line_by_line_section/chapter-comparison-results-all-rouge-1n-lbl.parquet", engine='pyarrow')

In [4]:
custom_ids = {} #silly but it works, ordered + hashed
for index, row in df.iterrows():
    if row['Reference Source'] == source:
        custom_id = row['Section Title'] + ", sentence-" + str(row['Reference Sentence Index'])
        custom_ids[custom_id] = custom_id
    # if len(custom_ids) >= 10:
    #     break

# print("Doneish")

for e in custom_ids.keys():
    print(e)

bleak-house-chapter-1, sentence-0
bleak-house-chapter-1, sentence-1
bleak-house-chapter-1, sentence-2
bleak-house-chapter-1, sentence-3
bleak-house-chapter-1, sentence-4
bleak-house-chapter-1, sentence-5
bleak-house-chapter-1, sentence-6
bleak-house-chapter-1, sentence-7
bleak-house-chapter-1, sentence-8
bleak-house-chapter-1, sentence-9
bleak-house-chapter-1, sentence-10
bleak-house-chapter-1, sentence-11
bleak-house-chapter-1, sentence-12
bleak-house-chapter-1, sentence-13
bleak-house-chapter-1, sentence-14
bleak-house-chapter-1, sentence-15
bleak-house-chapter-1, sentence-16
bleak-house-chapter-2, sentence-0
bleak-house-chapter-2, sentence-1
bleak-house-chapter-2, sentence-2
bleak-house-chapter-2, sentence-3
bleak-house-chapter-2, sentence-4
bleak-house-chapter-2, sentence-5
bleak-house-chapter-2, sentence-6
bleak-house-chapter-2, sentence-7
bleak-house-chapter-2, sentence-8
bleak-house-chapter-2, sentence-9
bleak-house-chapter-2, sentence-10
bleak-house-chapter-2, sentence-11
bleak

In [5]:

sources = ["bookwolf", "cliffnotes", "gradesaver", "novelguide", "pinkmonkey", "sparknotes", "thebestnotes"]

newdf = pd.DataFrame(index = custom_ids.keys(), columns=sources)

print(newdf.head)

for index, row in df.iterrows():
    if row['Reference Source'] == source:
        custom_id = row['Section Title'] + ", sentence-" + str(row['Reference Sentence Index'])
        if np.isnan(newdf.loc[custom_id, row["Hypothesis Source"]]):
            newdf.loc[custom_id, row["Hypothesis Source"]] = row['rouge-1nscore']
        newdf.loc[custom_id, row["Hypothesis Source"]] = max(newdf.loc[custom_id, row["Hypothesis Source"]], row['rouge-1nscore'])

newdf = newdf.fillna(value= np.nan)
newdf.to_csv("../csv_results/kappa_results/shmoop-section-rouge1n-sentence-max-scores.csv", na_rep=np.nan, index=False)

<bound method NDFrame.head of                                   bookwolf cliffnotes gradesaver novelguide  \
bleak-house-chapter-1, sentence-0      NaN        NaN        NaN        NaN   
bleak-house-chapter-1, sentence-1      NaN        NaN        NaN        NaN   
bleak-house-chapter-1, sentence-2      NaN        NaN        NaN        NaN   
bleak-house-chapter-1, sentence-3      NaN        NaN        NaN        NaN   
bleak-house-chapter-1, sentence-4      NaN        NaN        NaN        NaN   
...                                    ...        ...        ...        ...   
uncle-vanya-act-4, sentence-12         NaN        NaN        NaN        NaN   
uncle-vanya-act-4, sentence-13         NaN        NaN        NaN        NaN   
uncle-vanya-act-4, sentence-14         NaN        NaN        NaN        NaN   
uncle-vanya-act-4, sentence-15         NaN        NaN        NaN        NaN   
uncle-vanya-act-4, sentence-16         NaN        NaN        NaN        NaN   

                     

In [3]:
metdf = pd.read_csv("../csv_results/kappa_results/shmoop-section-rouge1n-sentence-max-scores.csv",index_col = 0)
display(metdf.describe())
print(metdf.head(10))
mets = metrics.Metrics(metdf)
fleiss = mets.fleiss_kappa()
print("Fleiss kappa: {:.2f}".format(fleiss))

,bookwolf,cliffnotes,gradesaver,novelguide,pinkmonkey,sparknotes,thebestnotes
count,3863.000000,28922.000000,33609.000000,32036.000000,21063.000000,38597.000000,6391.000000
mean,0.207485,0.214758,0.233026,0.227715,0.229454,0.223684,0.230319
std,0.099377,0.096207,0.099195,0.096899,0.096802,0.097988,0.097123
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.148148,0.157895,0.173913,0.166667,0.173913,0.166667,0.173913
50%,0.210526,0.214286,0.230769,0.222222,0.228571,0.222222,0.227273
75%,0.266667,0.271186,0.285714,0.285714,0.285714,0.279070,0.285714
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.923077


/home/jake/CodyStuff/disagree/disagree/utils.py:66: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, col in df.iteritems():


Fleiss kappa: 0.03


In [4]:
print(metdf.head(10))
kripp = metrics.Krippendorff(metdf)
alpha = kripp.alpha(data_type="ratio")
print("Krippendorff's alpha: {:.2f}".format(alpha))

                                   bookwolf  cliffnotes  gradesaver  \
bleak-house-chapter-1, sentence-0       NaN    0.108108         NaN   
bleak-house-chapter-1, sentence-1       NaN    0.206897         NaN   
bleak-house-chapter-1, sentence-2       NaN    0.232558         NaN   
bleak-house-chapter-1, sentence-3       NaN    0.250000         NaN   
bleak-house-chapter-1, sentence-4       NaN    0.117647         NaN   
bleak-house-chapter-1, sentence-5       NaN    0.153846         NaN   
bleak-house-chapter-1, sentence-6       NaN    0.187500         NaN   
bleak-house-chapter-1, sentence-7       NaN    0.171429         NaN   
bleak-house-chapter-1, sentence-8       NaN    0.384615         NaN   
bleak-house-chapter-1, sentence-9       NaN    0.145455         NaN   

                                   novelguide  pinkmonkey  sparknotes  \
bleak-house-chapter-1, sentence-0    0.153846         NaN    0.117647   
bleak-house-chapter-1, sentence-1    0.187500         NaN    0.200000   

/home/jake/CodyStuff/disagree/disagree/utils.py:66: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, col in df.iteritems():


Krippendorff's alpha: 0.19


In [5]:
metdf = pd.read_csv("../csv_results/kappa_results/shmoop-section-rouge1n-sentence-max-scores-threshold-adjusted.csv", index_col=0)
display(metdf.describe())
print(metdf.head(10))
mets = metrics.Metrics(metdf)
fleiss = mets.fleiss_kappa()
print("Fleiss kappa: {:.2f}".format(fleiss))

,bookwolf,cliffnotes,gradesaver,novelguide,pinkmonkey,sparknotes,thebestnotes
count,3863.000000,28922.000000,33609.000000,32036.000000,21063.000000,38597.000000,6391.000000
mean,0.531452,0.557430,0.634265,0.606661,0.621279,0.593388,0.614771
std,0.499074,0.496699,0.481643,0.488499,0.485080,0.491208,0.486688
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


                                   bookwolf  cliffnotes  gradesaver  \
bleak-house-chapter-1, sentence-0       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-1       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-2       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-3       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-4       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-5       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-6       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-7       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-8       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-9       NaN         0.0         NaN   

                                   novelguide  pinkmonkey  sparknotes  \
bleak-house-chapter-1, sentence-0         0.0         NaN         0.0   
bleak-house-chapter-1, sentence-1         0.0         NaN         0.0   

/home/jake/CodyStuff/disagree/disagree/utils.py:66: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, col in df.iteritems():


Fleiss kappa: -0.01


In [6]:
print(metdf.head(10))
kripp = metrics.Krippendorff(metdf)
alpha = kripp.alpha(data_type="ratio")
print("Krippendorff's alpha: {:.2f}".format(alpha))

                                   bookwolf  cliffnotes  gradesaver  \
bleak-house-chapter-1, sentence-0       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-1       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-2       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-3       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-4       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-5       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-6       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-7       NaN         0.0         NaN   
bleak-house-chapter-1, sentence-8       NaN         1.0         NaN   
bleak-house-chapter-1, sentence-9       NaN         0.0         NaN   

                                   novelguide  pinkmonkey  sparknotes  \
bleak-house-chapter-1, sentence-0         0.0         NaN         0.0   
bleak-house-chapter-1, sentence-1         0.0         NaN         0.0   

/home/jake/CodyStuff/disagree/disagree/utils.py:66: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, col in df.iteritems():


Krippendorff's alpha: 0.45
